In [ ]:
import os
import numpy as np

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Activation, Flatten, Reshape, Lambda, Input, concatenate
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, UpSampling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
K=tf.keras.backend


In [ ]:
#Por simplicidad usamos media y varianza solo del primer archivo
Datos=np.load('Datos_P/Datos_'+ str(0)+'.npy')
Media=np.mean(Datos, axis=0)
Varianza=np.var(Datos, axis=0)
# np.save('Media', Media)
# np.save('Varianza', Varianza)

In [ ]:
M=1
EstimaPar= Sequential()
EstimaPar.add( Conv2D(M*4, (3,3), strides=(2,2), activation='relu', padding='same' , input_shape=(512,512,3) ) ) #256x256x6
EstimaPar.add( Conv2D(M*8, (3,3), strides=(2,2), activation='relu', padding='same') ) # 128x128x12
EstimaPar.add( Conv2D(M*16, (3,3), strides=(2,2), activation='relu', padding='same') ) # 64x64x24
EstimaPar.add( Conv2D(M*32, (3,3), strides=(2,2), activation='relu', padding='same') ) # 32x32x48
EstimaPar.add( Conv2D(M*64, (3,3), strides=(2,2), activation='relu', padding='same') ) # 16x16x92
EstimaPar.add( Conv2D(M*128, (3,3), strides=(2,2), activation='relu', padding='same') ) # 8x8x92
EstimaPar.add( Conv2D(M*256, (3,3), strides=(2,2), activation='relu', padding='same') ) # 4x4x256
EstimaPar.add(Flatten())
EstimaPar.add(Dense(128*M, activation='tanh'))
EstimaPar.add(Dense(64*M, activation='tanh'))
EstimaPar.add(Dense(3*3*4, activation='linear'))
EstimaPar.add(Reshape((4,3,3)))

In [ ]:
EstimaPar.summary()

In [ ]:
LR=0.001

optimizer0 = RMSprop(learning_rate=LR, epsilon=1e-10)
EstimaPar.compile(loss='mse', optimizer=optimizer0, metrics='mse')


In [ ]:
import gc
gc.collect()

In [ ]:
EstimaPar.trainable=True

## Entrenamiento

In [ ]:
DatosOut=np.zeros( (18000,512,512,3), dtype='float32')

In [ ]:

for z in range(0,10):
    for it in range(0,100):

        DatosOut[:,:,:,0:1]=np.load('Datos_P/Datos_Out_'+ str(it)+'.npz')['arr_0']
        DatosOut[:,:,:,1]=DatosOut[:,:,:,0]//2
        DatosOut[:,:,:,2]=DatosOut[:,:,:,1]//2
        DatosOut[:,:,:,0]=DatosOut[:,:,:,0]-DatosOut[:,:,:,1]*2
        DatosOut[:,:,:,1]=DatosOut[:,:,:,1]-DatosOut[:,:,:,2]*2

        Datos=np.load('Datos_P/Datos_'+ str(it)+'.npy')

        gc.collect()

        Datos=(Datos-Media)/np.sqrt(Varianza)

        for it2 in range(0,25):
            Hist=EstimaPar.fit( DatosOut[np.mod(it2,25)::25,:,:,:], Datos[np.mod(it2,25)::25,:,:,:], batch_size=20, epochs=1, verbose=True)
            print(it2,1-Hist.history['loss'][0],LR)
            LR=LR*0.999
            K.set_value(EstimaPar.optimizer.lr,LR)
            gc.collect()

## Evaluación

In [ ]:
###########################################
# Datos de entrada
############################################
DatosOut_Test=np.zeros( (1000,512,512,3), dtype='float32')

DatosOut_Test[:,:,:,0:1]=np.load('Datos_Out_Test_0.npz')['arr_0'][0:1000,:,:,:]
DatosOut_Test[:,:,:,1]=DatosOut_Test[:,:,:,0]//2
DatosOut_Test[:,:,:,2]=DatosOut_Test[:,:,:,1]//2
DatosOut_Test[:,:,:,0]=DatosOut_Test[:,:,:,0]-DatosOut_Test[:,:,:,1]*2
DatosOut_Test[:,:,:,1]=DatosOut_Test[:,:,:,1]-DatosOut_Test[:,:,:,2]*2

In [ ]:
###########################################
# Datos de salida
###########################################
Datos_A_Estimar = np.load('Datos_Test_0.npy')
#Datos_A_Estimar = np.load('Datos_Test_1.npy')

Datos_A_Estimar= (Datos_A_Estimar-Media)/np.sqrt(Varianza)

In [ ]:

#Formato: (4,3,3) => 4 sensores, 3 carriles, 3 variables
###########################################


###########################################
# Estimación
pr = EstimaPar.predict(DatosOut_Test)






In [ ]:
Datos_A_Estimar.shape

In [ ]:
Datos_A_Estimar= ((Datos_A_Estimar)*np.sqrt(Varianza))+Media
pr= ((pr)*np.sqrt(Varianza))+Media

Gráficas por variable y dentro los 3 carriles

Flujo

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.plot(Datos_A_Estimar[0:1000,0,0,0],pr[0:1000,0,0,0], '*', label = 'Carril 1') #Carril 0
ax.plot(Datos_A_Estimar[0:1000,0,1,0],pr[0:1000,0,1,0], '*', label = 'Carril 2')#Carril 1
ax.plot(Datos_A_Estimar[0:1000,0,2,0],pr[0:1000,0,2,0], '*', label = 'Carril 3')#Carril 2
ax.plot([min(Datos_A_Estimar[0:1000,0,0,0]), max(Datos_A_Estimar[0:1000,0,0,0])], [min(Datos_A_Estimar[0:1000,0,0,0]), max(Datos_A_Estimar[0:1000,0,0,0])], 'k')
ax.legend()
ax.set_title('Flujo (vh/s)')
plt.xlabel('Q')
plt.ylabel( 'Q^')
#plt.savefig("ComparativaFlujoCarriles.pdf")
plt.show()

Densidad

In [ ]:
fig, ax = plt.subplots()
ax.plot(Datos_A_Estimar[0:1000,0,0,1],pr[0:1000,0,0,1], '*', label = 'Carril 1') #Carril 0
ax.plot(Datos_A_Estimar[0:1000,0,1,1],pr[0:1000,0,1,1], '*', label = 'Carril 2')#Carril 1
ax.plot(Datos_A_Estimar[0:1000,0,2,1],pr[0:1000,0,2,1], '*', label = 'Carril 3')#Carril 2
ax.plot([min(Datos_A_Estimar[0:1000,0,0,1]), max(Datos_A_Estimar[0:1000,0,0,1])], [min(Datos_A_Estimar[0:1000,0,0,1]), max(Datos_A_Estimar[0:1000,0,0,1])], 'k')
ax.legend()
ax.set_title('Densidad (vh/m)')
plt.xlabel('K')
plt.ylabel('K^')
#plt.savefig("ComparativaDensidadCarriles.pdf")
plt.show()

Velocidad

In [ ]:
fig, ax = plt.subplots()
ax.plot(Datos_A_Estimar[0:1000,0,0,2],pr[0:1000,0,0,2], '*', label = 'Carril 1') #Carril 0
ax.plot(Datos_A_Estimar[0:1000,0,1,2],pr[0:1000,0,1,2], '*', label = 'Carril 2')#Carril 1
ax.plot(Datos_A_Estimar[0:1000,0,2,2],pr[0:1000,0,2,2], '*', label = 'Carril 3')#Carril 2
ax.plot([min(Datos_A_Estimar[0:1000,0,2,2]), max(Datos_A_Estimar[0:1000,0,0,2])], [min(Datos_A_Estimar[0:1000,0,2,2]), max(Datos_A_Estimar[0:1000,0,0,2])], 'k')
ax.legend()
ax.set_title('Velocidad (m/s)')
plt.xlabel('V')
plt.ylabel('V^')
#plt.savefig("ComparativaVelocidadCarriles.pdf")
plt.show()

Intento de dibujar gráficas con valores

Flujo

In [ ]:
fig, ax = plt.subplots()
ax.plot(Datos_A_Estimar[0:200,0,0,0], '*')
ax.plot(pr[0:200,0,0,0],'*')
plt.show()

Densidad

In [ ]:
fig, ax = plt.subplots()
ax.plot(Datos_A_Estimar[0:200,0,0,1], '*')
ax.plot(pr[0:200,0,0,1],'*')
plt.show()

Velocidad

In [ ]:
fig, ax = plt.subplots()
ax.plot(Datos_A_Estimar[0:200,0,0,2], '*')
ax.plot(pr[0:200,0,0,2],'*')
plt.show()

In [ ]:
###########################################
carril = 2
variable = 2

#ErrorCuadraticoMedio
MSE=0
MSE = np.mean(np.power(Datos_A_Estimar[0:1000,0,carril,variable]-pr[:,0,carril,variable],2))
#R^2
R2=0
R2 = 1 - np.var(Datos_A_Estimar[0:1000,0,carril,variable]-pr[0:1000,0,carril,variable])/np.var(Datos_A_Estimar[0:1000,0,carril,variable])





In [ ]:
print(MSE)

In [ ]:
print(R2)